<a href="https://colab.research.google.com/github/abbeymars/AbigailMarsella-AER850Project2/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from keras import layers

#Step 1: Data Processing
#1.1 Define input image shape
input_shape = (500, 500, 3)  # Image dimensions: 500x500 pixels, 3 color channels (RGB)
batch_size = 32

#1.2 Establish data directories
from google.colab import drive
drive.mount('/content/drive') #mount google drive
zip_file = '/content/drive/My Drive/AER850Proj2/Dataset.zip'
extract = '/content/Dataset/Data'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract) #extract data
#establish train, validation, and test directories
train = os.path.join(extract, 'train')
validation = os.path.join(extract, 'valid')
test = os.path.join(extract, 'test')
#verify data paths are set correctly
print("Training directory:", train)
print("Validation directory:", validation)
print("Test directory:", test)

#1.3 Perform Data Augementation
# Training data
train_data = ImageDataGenerator(
    rescale=1/255, #scale pixel values from [0, 255] to [0, 1]
    shear_range=0.2, #shear intensity
    zoom_range=0.2, #zoom range
    horizontal_flip=True #randomly flip images horizontally
)
# Validation data (note: apply only rescaling for validation)
validation_data = ImageDataGenerator(
    rescale=1/255 #scale pixel values from [0, 255] to [0, 1]
)

#1.4 Create the Train and Validation Generators
#train generator
train_generator = train_data.flow_from_directory(
    train,
    target_size=(500, 500),      #resize images (500x500 pixels)
    batch_size=batch_size,       #batch size 32
    class_mode='categorical',    #class mode categorical
    shuffle=True                 #shuffle images, ensure randomness
)
#validation generator
validation_generator = validation_data.flow_from_directory(
    validation,
    target_size=(500, 500),      #resize images (500x500 pixels)
    batch_size=batch_size,       # batch size 32
    class_mode='categorical',    # class mode categorical
    shuffle=False                # no shuffle, doesn't need to be random
)
#verify generators are set up correctly
print("Class indices:", train_generator.class_indices)
print("Class indices:", validation_generator.class_indices)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training directory: /content/Dataset/Data/train
Validation directory: /content/Dataset/Data/valid
Test directory: /content/Dataset/Data/test
Found 1942 images belonging to 3 classes.
Found 431 images belonging to 3 classes.
Class indices: {'crack': 0, 'missing-head': 1, 'paint-off': 2}
Class indices: {'crack': 0, 'missing-head': 1, 'paint-off': 2}
